في هذا النشاط سنصنف المدن بناء على المحتوى وبالعمل على عينة من التغريدات. وهي دراسة الحالة نفسها التي عملنا عليها في النشاط 2.2. ولكننا في المرة الأخيرة استعملنا مصنف انحدار لوجيستي بالاستعانة بـ *scikit-learn*. غير أن هذه المرة سنتناول نفس المشكلة ولمن باستعمال مصنف MLP (شبكة المستقبلات المتعددة الطبقات multi-layer perceptron) بالاستعانة بـ *tensorflow*.
وكالعادة هذا النشاط يبين النسخة البسيطة الممكنة، فإذا ما أردت الاستزادة يمكنك العودة إلى الأمثلة في حزمة *text_analytics()*.


In [1]:
from text_analytics import TextAnalytics
import os
import pandas as pd

ai = TextAnalytics()
ai.data_dir = os.path.join("..", "data")
print("Done!")

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Done!


لنرفع الآن التغريدات التي نحتاجها في كل مدينة. وسنقلل من حجم المدونة لنتجنب عناء الإشكاليات التي تتعلق بالذاكرة.

In [2]:
file = "sociolinguistics.english_cities.gz"
file = os.path.join(ai.data_dir, file)
df = pd.read_csv(file, nrows=65000)
print(df)
print("Done!")

             City                                               Text
0      washington   you really need to go back to bar tending or ...
1          london   jay finley christ in explains why today is co...
2           lagos   forget if this happened truly it s definitely...
3         toronto   yall i love this skin big thanks to for makin...
4         nairobi   the late brilliant prof ali mazrui explains h...
...           ...                                                ...
64995     toronto   taxing issues federal government to charge gs...
64996   bengaluru   this is revealing of americans are held hosta...
64997      mumbai   pumps in pumping station did not work in my s...
64998     atlanta   why is it that some ppl can speak amp act w i...
64999     phoenix   you netflix in already ready for more seasons...

[65000 rows x 2 columns]
Done!


والآن رفعت بياناتنا. لنلق نظرة عامة على المدن مرة أخرى. 

In [3]:
freqs = ai.print_labels(df, "City")
for city in freqs:
    print(city, freqs[city])

washington 3263
london 4154
lagos 3057
toronto 3268
nairobi 2505
melbourne 1125
perth 946
chicago 3614
dallas 3013
chennai 1230
delhi 2506
mumbai 2608
atlanta 2787
bengaluru 2528
seattle 1046
johannesburg 2772
adelaide 968
miami 1990
calgary 1177
boston 3672
phoenix 1096
auckland 899
brisbane 1054
karachi 1282
new_york 3042
los_angeles 2500
san_francisco 3113
singapore 1743
kolkata 1273
syndney 769



إلى الآن لدينا (1) بيانات من كل مدينة و(2) محتوى متجهي (عبارات وأوزان TF-IDF من مدونات رقمية ضخمة). وسنصنف هذه العينات بحسب المدينة. وأدناه الكود البرمجي الأساسي، وفيه فقط استدعاء لحزمتنا *text_analytics*. وتلك الحزمة تقسم البيانات إلى بيانات تدريب وبيانات اختبار، ثم تدرب المصنف وتقيمه. وقد وجهنا الحزمة لاستعمال "المدينة City" صنفا أساسيا مع خصائص المحتوى.

ولننظر الآن في خصائص المفردات.


In [4]:
#ai.fit_phrases(df, min_count=1, language="en")
#ai.fit_tfidf(df, n_features = 10000)

file_phrases = os.path.join(ai.data_dir, "sociolinguistics.english_all.gz")
ai.phrases = ai.deserialize("phrases", file_phrases + ".phrases.json")
ai.tfidf_vectorizer = ai.deserialize("tfidf_model", file + ".tfidf.json")

print("PHRASES:")
example_phrases = list(ai.phrases.phrasegrams.keys())
print(example_phrases[:12])

print("\nVOCAB")
example_vocab = list(ai.tfidf_vectorizer.vocabulary_.keys())
print(example_vocab[:12])

PHRASES:
['united_states', 'boris_johnson', 'chuck_schumer', 'ricky_gervais', 'rashida_tlaib', 'hillary_clinton', 'devin_nunes', 'sherlock_holmes', 'edinson_cavani', 'climate_change', 'harry_potter', 'happy_birthday']

VOCAB
['people', 'new', 'today', 'need', 'want', 'video', 'life', 'world', 'year', 'please', 'watch', 'thanks']


وما هو مختلف هنا هو أننا في المرة الأخيرة استعملنا SVM، وهذه المرة سنستعمل MLP. وSVM تتدرب على كل شيء مرة واحدة، فيما تتدرب MLP مع مرور الزمن؛ ولذلك سنتمكن من تتبع النموذج أثناء تعلمه.

In [5]:
report = ai.mlp(df, labels = "City", features = "content", layers = [500, 500])
print(report)

Epoch 1/25
1829/1829 [==============================] - 35s 18ms/step - loss: 0.4933 - accuracy: 0.8494 - val_loss: 0.1701 - val_accuracy: 0.9440
Epoch 2/25
1829/1829 [==============================] - 32s 17ms/step - loss: 0.0420 - accuracy: 0.9875 - val_loss: 0.2116 - val_accuracy: 0.9374
Epoch 3/25
1829/1829 [==============================] - 33s 18ms/step - loss: 0.0172 - accuracy: 0.9945 - val_loss: 0.2840 - val_accuracy: 0.9243
Epoch 4/25
1829/1829 [==============================] - 33s 18ms/step - loss: 0.0170 - accuracy: 0.9945 - val_loss: 0.2946 - val_accuracy: 0.9328
Epoch 5/25
1829/1829 [==============================] - 25s 14ms/step - loss: 0.0132 - accuracy: 0.9955 - val_loss: 0.3180 - val_accuracy: 0.9292
Epoch 6/25
1829/1829 [==============================] - 25s 14ms/step - loss: 0.0112 - accuracy: 0.9960 - val_loss: 0.3685 - val_accuracy: 0.9265
Epoch 7/25
1829/1829 [==============================] - 22s 12ms/step - loss: 0.0098 - accuracy: 0.9967 - val_loss: 0.3816 -

**طوّل بالك**

هذا كل ما في الأمر! ننظر في دقة المصنف.
وسيتغير ذلك عما في المبحث قليلا لأننا نستعمل فصلا عشوائيا لبيانات التدريب والاختبار. وهذا يعني أن المصنف ينظر في بيانات مختلفة كل مرة. وإذا ما أردت أمثلة إضافية متقدمة على حل إشكالية تصنيف المدن، انظر في دالة *text_analytics.mlp()*.
